In [7]:

import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection 

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

2023-04-09 06:04:18.814523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'sklearn'

In [10]:

!pip3 install scikit-learn


  Using cached scikit_learn-1.0.2-cp37-cp37m-macosx_10_13_x86_64.whl (7.8 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.7.3-cp37-cp37m-macosx_10_9_x86_64.whl (33.0 MB)


In [11]:
# Download model from TF Hub and check out inference code from GitHub

!git clone https://github.com/tensorflow/examples.git
pose_sample_rpi_path = os.path.join(os.getcwd(), 'examples/lite/examples/pose_estimation/raspberry_pi')
sys.path.append(pose_sample_rpi_path)

# Load MoveNet Thunder model
import utils
from data import BodyPart
from ml import Movenet
movenet = Movenet('movenet_thunder')

# Define function to run pose estimation using MoveNet Thunder.
# You'll apply MoveNet's cropping algorithm and run inference multiple times on
# the input image to improve pose estimation accuracy.
def detect(input_tensor, inference_count=3):
  """Runs detection on an input image.

  Args:
    input_tensor: A [height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.
    inference_count: Number of times the model should run repeatly on the
      same input image to improve detection accuracy.

  Returns:
    A Person entity detected by the MoveNet.SinglePose.
  """
  image_height, image_width, channel = input_tensor.shape

  # Detect pose using the full input image
  movenet.detect(input_tensor.numpy(), reset_crop_region=True)

  # Repeatedly using previous detection result to identify the region of
  # interest and only croping that region to improve detection accuracy
  for _ in range(inference_count - 1):
    person = movenet.detect(input_tensor.numpy(), 
                            reset_crop_region=False)

  return person

Cloning into 'examples'...
remote: Enumerating objects: 23066, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 23066 (delta 6), reused 7 (delta 0), pack-reused 23038
Receiving objects: 100% (23066/23066), 42.17 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (12655/12655), done.
Updating files: 100% (2762/2762), done.


In [12]:
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.

  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.

  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])

  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)

  if close_figure:
    plt.close(fig)

  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [3]:
#@title Functions to visualize the pose estimation results.

def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [15]:

is_skip_step_1 = False #@param ["False", "True"] {type:"raw"}


class MoveNetPreprocessor1(object):
    """Helper class to preprocess pose sample images for classification."""

    def __init__(self, images_out_folder, csv_out_path):
        """Creates a preprocessor to detect pose from a single image and save the result as a CSV.

        Args:
            images_out_folder: Path to write the image overlay with detected landmarks. This image is useful when you need to debug accuracy issues.
            csv_out_path: Path to write the CSV containing the detected landmark coordinates and label of the image that can be used to train a pose classification model.
        """
        self._images_out_folder = images_out_folder
        self._csv_out_path = csv_out_path
        self._messages = []

    def process(self, image_path, detection_threshold=0.1):
        """Preprocesses a single image.

        Args:
            image_path: Path to the input image.
            detection_threshold: Only keep the image if all landmark confidence scores are above this threshold.
        """
        # Detect landmarks in the image and write it to a CSV file
        with open(self._csv_out_path, 'w') as csv_out_file:
            csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

            try:
                image = tf.io.read_file(image_path)
                image = tf.io.decode_jpeg(image)
            except:
                self._messages.append('Skipped ' + image_path + '. Invalid image.')
            else:
                image_height, image_width, channel = image.shape

                # Skip images that aren't RGB because Movenet requires RGB images
                if channel != 3:
                    self._messages.append('Skipped ' + image_path + '. Image isn\'t in RGB format.')
                else:
                    person = detect(image)

                    # Save landmarks if all landmarks were detected
                    min_landmark_score = min([keypoint.score for keypoint in person.keypoints])
                    should_keep_image = min_landmark_score >= detection_threshold
                    if not should_keep_image:
                        self._messages.append('Skipped ' + image_path + '. No pose was confidently detected.')
                    else:
                        # Draw the prediction result on top of the image for debugging later
                        output_overlay = draw_prediction_on_image(
                            image.numpy().astype(np.uint8), person, close_figure=True, keep_input_size=True)

                        # Write detection result into an image file
                        output_frame = cv2.cvtColor(output_overlay, cv2.COLOR_RGB2BGR)
                        filename = os.path.basename(image_path)
                        cv2.imwrite(os.path.join(self._images_out_folder, filename), output_frame)

                        # Get landmarks and scale them to the same size as the input image
                        pose_landmarks = np.array(
                            [[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score] for keypoint in person.keypoints],
                            dtype=np.float32)

                        # Write the landmark coordinates to the CSV file
                        coordinates = pose_landmarks.flatten().astype(str).tolist()
                        csv_out_writer.writerow([os.path.basename(image_path)] + coordinates)

        if not os.path.exists(self._images_out_folder):
            os.makedirs(self._images_out_folder)

        # Print the error message collected
        for message in self._messages:
            print(message, file=sys.stderr)

if not is_skip_step_1:
  images_out_train_folder = 'abc'
  csvs_out_train_path = 'train_data1.csv'

  preprocessor = MoveNetPreprocessor1(
      
      images_out_folder=images_out_train_folder,
      csv_out_path=csvs_out_train_path,
  )

  preprocessor.process('yoga_poses/train/tree/girl1_tree096.jpg')
      


 

